## Comment construire un nouveau Tokenizer ?

In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")


def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

In [ ]:
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, trainers, processors, decoders

In [ ]:
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [ ]:
tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace(Regex(r"[\p{Other}&&[^\n\t\r]]"), ""),
        normalizers.Replace(Regex(r"[\s]"), " "),
        normalizers.Lowercase(),
        normalizers.NFD(), normalizers.StripAccents()]
)

In [ ]:
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

In [ ]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

In [ ]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [ ]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [ ]:
tokenizer.decoder = decoders.WordPiece(prefix="##")